In [ ]:
import pandas as pd
import json
import pyomo.environ as pyo
from pyomo.util.infeasible import *
import networkx as nx
import matplotlib.pyplot as plt

## Data 

In [ ]:
with open("../data/mock.json") as f:
    d = json.load(f)
nodes, channels = pd.DataFrame(d["nodes"]), pd.DataFrame(d["edges"])

In [ ]:
G = nx.DiGraph()

# Add nodes to the graph
for _, node in nodes.iterrows():
    G.add_node(node['pub_key'], alias=node['alias'])

# Add edges to the graph
for _, edge in channels.iterrows():
    G.add_edge(edge['node1_pub'], edge['node2_pub'], capacity=edge['capacity'], base_fee=edge['base_fee'], perc_fee=edge['perc_fee'])

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_color='black', font_weight='bold')

# Add edge labels
edge_labels = {(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title('Network Graph')
plt.show()

## Model

In [ ]:
#channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.set_index("channel_id", inplace=True)
nodes.set_index("pub_key", inplace=True)

### Sets

In [ ]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=channels.index) #within=model.NODES*model.NODES)

### Variables

In [ ]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)

In [ ]:
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

### Objective function

$$\min \sum _{(i,j) \in E} x_{i,j} \times (baseFee_{i,j} + rateFee_{i,j} \times amount_{i,j})$$

In [ ]:
def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[k]["base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i]["perc_fee"] * model.a[i] for i in model.CHANNELS)

In [ ]:
model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$

In [ ]:
# Capacity constrain
def capacity_constraint(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a]["capacity"] * model.x[a]
model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$

In [ ]:
def compute_outgoing(n: str) -> list:
    """
    Compute outgoing channels list for the node n
    :param n: node identifier
    :return: list of outgoing channels for node n
    """
    return [c for c in model.CHANNELS if channels.loc[c]["node1_pub"] == n]


def compute_incoming(n: str) -> list:
    """
    Compute incoming channels list for the node n
    :param n: node identifier
    :return: list of incoming channels for node n
    """
    return [c for c in model.CHANNELS if channels.loc[c]["node2_pub"] == n]


In [ ]:
# Flow constrain

def flow_balance_constraint(model: pyo.ConcreteModel, n):
    return sum(model.a[a] for a in compute_incoming(n)) - sum(model.a[a] for a in compute_outgoing(n)) == nodes.loc[n]["demand"]
model.FlowBalanceConstrain = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

## Solving the model

In [ ]:
opt = pyo.SolverFactory('cbc')
opt.solve(model, tee=True)

In [ ]:
model.x.pprint()